In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
import pydicom as dcm
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import os
import math
from PIL import Image

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
%matplotlib inline

In [3]:
inception = keras.applications.resnet.ResNet101(include_top=False, input_shape=(512,512,3), weights='imagenet', 
                                                       classes=6)

2021-12-20 07:01:52.860527: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-20 07:01:53.430911: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38444 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0


In [4]:
len(inception.layers)

345

In [5]:
def get_center_and_width(dicom):
    return tuple([int(x[0]) if type(x) == dcm.multival.MultiValue else int(x) for x in [dicom.WindowCenter, dicom.WindowWidth]])
def normalize_minmax(img):
    mi, ma = img.min(), img.max()
    return (img - mi) / (ma - mi)

def window_filter(img, center, width, slope, intercept):
    out = np.copy(img)
    out = out*slope + intercept
    lowest_visible = center - width//2
    highest_visible = center + width//2
    
    out[out < lowest_visible] = lowest_visible
    out[out > highest_visible] = highest_visible
    return normalize_minmax(out)
def standardize(img):
    m, std = img.mean(), img.std()
    return (img - m) / std


In [14]:
model = keras.models.Sequential()
model.add(inception)
model.add(GlobalAveragePooling2D())
model.add(Dense(6))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet101 (Functional)      (None, 16, 16, 2048)      42658176  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 6)                 12294     
                                                                 
Total params: 42,670,470
Trainable params: 42,565,126
Non-trainable params: 105,344
_________________________________________________________________


In [16]:
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=['binary_accuracy'],
             optimizer=keras.optimizers.Adam(learning_rate=1e-4))
cp_callback = keras.callbacks.ModelCheckpoint(filepath='reproduce_training/checkpoint.ckpt',
                                                 save_weights_only=True,
                                                 verbose=1)

In [17]:
# data_tens = tf.convert_to_tensor(data)
# model.predict(data_tens)

In [18]:
# def get_img_tensor(img_path):
#     dicom = dcm.dcmread(img_path, force=True)
    
#     img = dicom.pixel_array
#     center, width = get_center_and_width(dicom)
#     slope, intercept = dicom.RescaleSlope, dicom.RescaleIntercept
#     brain = window_filter(img, 40, 80, slope, intercept)
#     subdural = window_filter(img, 80, 200, slope, intercept)
#     tissue = window_filter(img, 40, 380, slope, intercept)
    
#     return tf.convert_to_tensor(np.stack([brain, subdural, tissue], axis=2), dtype=tf.dtypes.float32)
def get_img_tensor(img_path):
    return tf.convert_to_tensor(np.asarray(Image.open(img_path), dtype=np.float32) / 255.)

In [19]:
class RSNASequence(keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x = x_set
        self.y = y_set
        self.batch_size = batch_size
    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = [self.y[img_id.split('/')[-1].split('.')[0]] for img_id in batch_x]
        # batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        return (tf.stack([get_img_tensor(img_path) for img_path in batch_x], axis=0), 
               tf.convert_to_tensor(batch_y))

In [20]:
png_path = 'rsna-intracranial-hemorrhage-detection/stage_2_train_imgs/'
labels = pd.read_csv('rsna-intracranial-hemorrhage-detection/train_labels.csv')
labels = {l[0]: l[1:].astype(np.int8) for l in labels.to_numpy()}

In [21]:
train_cutoff = 12800 #training the whole dataset takes ~9 hours, so we cut it short for proof-of-concept purposes.
train_sequence = RSNASequence([png_path + img_name for img_name in os.listdir(png_path)[:train_cutoff]], labels, 32)

In [22]:
model.fit(x=train_sequence, epochs=2, callbacks=[cp_callback])
# model.predict(train_sequence[1][0])
# train_sequence[1][0]

Epoch 1/2


2021-12-20 07:02:13.111712: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8200
2021-12-20 07:02:16.036751: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


400/400 [==============================] - ETA: 0s - loss: 0.1352 - binary_accuracy: 0.9533
Epoch 00001: saving model to reproduce_training/checkpoint.ckpt
400/400 [==============================] - 561s 1s/step - loss: 0.1352 - binary_accuracy: 0.9533
Epoch 2/2
400/400 [==============================] - ETA: 0s - loss: 0.0966 - binary_accuracy: 0.9643
Epoch 00002: saving model to reproduce_training/checkpoint.ckpt
400/400 [==============================] - 228s 570ms/step - loss: 0.0966 - binary_accuracy: 0.9643


We can see that training on just a few thousand of the 700k+ DICOM training images yields a 96% accuracy on the training dataset, using ResNet101 pretrained weights as a base and fine tuning using a custom softmax binary cross-entropy layer. These preliminary results are very promising, however it is unknown whether the model can generalize to the entire training set let alone the test set. However, I believe that I have succeeded in reproducing one part of the research that I have conducted- all solutions I found used a CNN with pretrained weights as the first stage for their model. While reproducing the entirety of their research would take up to a week of coding, debugging, and training (not to mention money); I believe that my results are sufficient for this capstone submission (which predicted a 5-20 hour timeframe). 

When testing full models, I plan to start by training at least 1 epoch on the full dataset with the CNN, then expirement with feeding the results and the extracted features (taken from the CNN's pennultimate layer) to feed into:
1. An ensemble method of boosting or bagging tree methods (as one of my sources did)
2. A RNN, perhaps a LSTM
3. Another CNN
4. An ensemble of RNNs and/or CNNs